In [ ]:
import re
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

In [7]:
df=pd.read_csv(r"C:\Users\Muhire Taa\Desktop\RAG\interview_dataset.csv")
df

C:\Users\Muhire Taa\AppData\Local\Temp\ipykernel_11928\1688135581.py:1: DtypeWarning: Columns (102,116,128,142,154,180,194,246,343,359,367,380,484,615,725,737,806,827,857,859,999,1076,1191,1234,1263,1538,1541,1592,1593,1632,1679,1681,1723,1735,1770,1785,1804,1855,1857,2182,2230,2252,2368,2577,2580,2631,2632,2671,2742,2758,2784,2805,2858,2885,2927,3364,3372,3510,3540,3543,3557,3571,3628,3741,3788,3835,3863,3882,3920,3930,4006,4044,4053,4100,4157,4383,4402,4430,4506,4553,4929,4943,5005,5009,5013,5017,5077,5166,5174,5177,5222,5491,5504,5537,5736,5745,5845,5849,5855,5866,5878,5879,5901,5923,5931,5945,5967,5995,5996,6011,6017,6018,6075,6115,6118,6430,6440,6451,6475,6483,6525,6532,6703,6763,6808) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\Muhire Taa\Desktop\RAG\interview_dataset.csv")


,SubmissionDate,starttime,endtime,version,duration,survey_start,intro_start,text_audit,date_collection,region,...,HH Production UGX,HH Income + Production UGX,Program Value UGX,Assets,HH Income UGX/Day,HH Production UGX/Day,HH Income + Production UGX/Day,Program Value UGX/Day,STATUS,HH Income + Production/Day (USD)
0,"Aug 6, 2024 3:52:21 PM","Aug 6, 2024 11:50:32 AM","Aug 6, 2024 12:44:45 PM",2408061513,3093,7.0,7.0,https://rtvuganda.surveycto.com/view/submissio...,2024-08-07 00:00:00,4,...,1825000.0,5253000.0,6148000.0,895000,9391.780822,5000.000000,14391.780822,16843.835616,RTV,3.889670
1,"Aug 1, 2024 5:44:43 PM","Aug 1, 2024 11:36:23 AM","Aug 1, 2024 12:13:08 PM",2407311236,2003,62.0,62.0,https://rtvuganda.surveycto.com/view/submissio...,2024-08-02 00:00:00,4,...,786000.0,1335000.0,1335000.0,0,1504.109589,2153.424658,3657.534247,3657.534247,RTV,0.988523
2,"Aug 3, 2024 12:34:58 AM","Aug 2, 2024 4:13:56 PM","Aug 2, 2024 5:02:05 PM",2408021241,2890,7.0,7.0,https://rtvuganda.surveycto.com/view/submissio...,2024-08-03 00:00:00,4,...,1955000.0,2385000.0,2535000.0,150000,1178.082192,5356.164384,6534.246575,6945.205479,RTV,1.766013
3,"Aug 6, 2024 9:17:47 PM","Aug 6, 2024 10:44:39 AM","Aug 6, 2024 12:10:57 PM",2408061513,5178,5.0,5.0,https://rtvuganda.surveycto.com/view/submissio...,2024-08-07 00:00:00,2,...,789000.0,3510500.0,6160500.0,2650000,7456.164384,2161.643836,9617.808219,16878.082192,RTV,2.599408
4,"Aug 1, 2024 1:39:03 PM","Aug 1, 2024 12:20:03 PM","Aug 1, 2024 1:27:07 PM",2407302157,4024,5.0,5.0,https://rtvuganda.surveycto.com/view/submissio...,2024-08-02,4,...,868350.0,2326350.0,3276350.0,950000,3994.520548,2379.041096,6373.561644,8976.301370,RTV,1.722584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,"Aug 1, 2024 6:21:55 PM","Aug 1, 2024 11:04:32 AM","Aug 1, 2024 12:00:03 PM",2407311236,3331,2.0,2.0,https://rtvuganda.surveycto.com/view/submissio...,2024-08-02,4,...,1061500.0,2645500.0,2715500.0,70000,4339.726027,2908.219178,7247.945205,7439.726027,RTV,1.958904
496,"Aug 7, 2024 9:10:44 PM","Aug 7, 2024 1:02:35 PM","Aug 7, 2024 2:06:50 PM",2408061513,3736,16.0,16.0,https://rtvuganda.surveycto.com/view/submissio...,2024-08-08,4,...,1455500.0,3335000.0,3945000.0,610000,5149.315068,3987.671233,9136.986301,10808.219178,RTV,2.469456
497,"Jul 30, 2024 7:22:54 PM","Jul 30, 2024 12:45:00 PM","Jul 30, 2024 3:53:20 PM",2407302157,3303,3.0,3.0,https://rtvuganda.surveycto.com/view/submissio...,2024-07-31 00:00:00,4,...,1960750.0,6156000.0,6201000.0,45000,11493.835616,5371.917808,16865.753425,16989.041096,RTV,4.558312
498,"Jul 29, 2024 7:41:46 PM","Jul 29, 2024 2:09:08 PM","Jul 29, 2024 3:26:55 PM",2407300828,4590,2.0,2.0,https://rtvuganda.surveycto.com/view/submissio...,2024-07-30 00:00:00,4,...,503000.0,1182000.0,1682000.0,500000,1860.273973,1378.082192,3238.356164,4608.219178,PEER,0.875231


In [8]:
df=df[['most_recommend_rtv_program','least_recommend_rtv_program','most_recommend_rtv_program_reason','least_recommend_rtv_program_reason']]

In [9]:
#check if there are nans
print(df.isna().any().any())

True


In [10]:
df.dropna(inplace=True)
df

C:\Users\Muhire Taa\AppData\Local\Temp\ipykernel_11928\955798070.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


,most_recommend_rtv_program,least_recommend_rtv_program,most_recommend_rtv_program_reason,least_recommend_rtv_program_reason
0,1.0,5.0,To improve household food security and nutriti...,Low saving capacity due to limited sources of ...
1,1.0,5.0,The active beneficiaries of the program were t...,The respondent didn't benefit from the program...
2,3.0,99.0,Water is highly recommended as we were given a...,All programs are recommended because water has...
3,4.0,3.0,Because if u are alive u do what u fill like a...,Since am surrounded by many water sources I co...
4,3.0,99.0,Ever since a clean and safe water source was i...,"Agricultural practices and seedlings, water so..."
...,...,...,...,...
493,1.0,3.0,The agricultural sector in these communities h...,Water is still a problem here in away that peo...
495,2.0,99.0,This has helped improve the hygiene of the vil...,All programs are very good and help the villag...
496,1.0,99.0,The respondent said that the program most reco...,The respondent said that all the programs of R...
497,3.0,99.0,The respondent recommend water because they we...,All the programs are good and highly recommend...


In [11]:
def clean_text(text):
    """
    Clean the text by converting to lowercase and removing non-alphanumeric characters.
    """
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text.strip()

def process_feedback_data(df):
    """
    Process the DataFrame to extract feedback entries from both reason fields.
    Each entry is a dictionary with:
      - 'program': The associated program name.
      - 'feedback': The explanation text.
      - 'type': 'positive' for most_recommend and 'negative' for least_recommend.
    """
    feedback_entries = []
    for _, row in df.iterrows():
        if pd.notnull(row['most_recommend_rtv_program_reason']):
            feedback_entries.append({
                'program': row['most_recommend_rtv_program'],
                'feedback': row['most_recommend_rtv_program_reason'],
                'type': 'positive'
            })
        if pd.notnull(row['least_recommend_rtv_program_reason']):
            feedback_entries.append({
                'program': row['least_recommend_rtv_program'],
                'feedback': row['least_recommend_rtv_program_reason'],
                'type': 'negative'
            })
    return feedback_entries

# Process our sample DataFrame
feedback_entries = process_feedback_data(df)
print("Processed Feedback Entries:")
for entry in feedback_entries:
    print(entry)


Processed Feedback Entries:
{'program': 1.0, 'feedback': 'To improve household food security and nutrition which has boosted individual ability to work more and expand agriculture which improves crop yields and hence household poverty reduction.', 'type': 'positive'}
{'program': 5.0, 'feedback': 'Low saving capacity due to limited sources of incomes and lack of individual trust in saving with Vslas due to associated with some groups in the village.', 'type': 'negative'}
{'program': 1.0, 'feedback': 'The active beneficiaries of the program were taught new and cost effective methods of farming for improved food security', 'type': 'positive'}
{'program': 5.0, 'feedback': "The respondent didn't benefit from the program at all. But it seems he was not an active caver and was not very ,uch involved in RTV business", 'type': 'negative'}
{'program': 3.0, 'feedback': 'Water is highly recommended as we were given a tank that has reduced distances for collecting water and VSLAS have brought in sa

In [ ]:
# Load the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

def generate_feedback_embeddings(feedback_entries, model):
    """
    Generate embeddings for each feedback entry using the provided Sentence Transformer model.
    The feedback text is cleaned before encoding.
    """
    texts = [clean_text(entry['feedback']) for entry in feedback_entries]
    embeddings = model.encode(texts)
    return np.array(embeddings).astype('float32')

def build_faiss_index(embeddings):
    """
    Build a FAISS index (using L2 distance) for efficient semantic search.
    """
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

# Generate embeddings and build the index
embeddings = generate_feedback_embeddings(feedback_entries, model)
index = build_faiss_index(embeddings)

print("FAISS index built with {} embeddings.".format(index.ntotal))


In [ ]:
def retrieve_feedback(query, model, index, feedback_entries, top_k=3):
    """
    Given a stakeholder query, clean and embed it, then retrieve the top_k most relevant feedback entries.
    """
    query_clean = clean_text(query)
    query_embedding = model.encode([query_clean]).astype('float32')
    distances, indices = index.search(query_embedding, top_k)
    retrieved = []
    for idx in indices[0]:
        if idx < len(feedback_entries):
            retrieved.append(feedback_entries[idx])
    return retrieved

#example query and retrieval
query_example = "What reasons do communities give for recommending Agriculture & Nutrition programs?"
retrieved_feedback = retrieve_feedback(query_example, model, index, feedback_entries)
print("Retrieved Feedback for Query:\n", query_example)
for fb in retrieved_feedback:
    print(f"[{fb['type'].capitalize()}] {fb['program']}: {fb['feedback']}")


In [ ]:
def generate_response(query, retrieved_feedback):
    """
    Combine the stakeholder query with the retrieved feedback entries into a prompt.
    This prompt is then (simulated) sent to a generative model to produce a final response.
    Replace the simulated response with a call to a generative model API (e.g., Gemini) in production.
    """
    prompt = f"Query: {query}\n\nRelevant community feedback:\n"
    for feedback in retrieved_feedback:
        prompt += f"- [{feedback['type']}] {feedback['program']}: {feedback['feedback']}\n"
    prompt += "\nBased on the above, provide a concise analysis."
    
    # Simulated response 
    response = (
        "Based on the retrieved feedback, Agriculture & Nutrition programs are appreciated for their strong support "
        "for local farmers, while challenges such as facility maintenance in WASH programs are frequently mentioned. "
        "Stakeholders might consider targeted improvements to address these issues."
    )
    return prompt, response

# Generate and display the response
prompt, chatbot_response = generate_response(query_example, retrieved_feedback)
print("Constructed Prompt for Generative Model:")
print(prompt)
print("\nSimulated Chatbot Response:")
print(chatbot_response)
